In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as tfl

## Personalized Convolutional Network
### First Attempt: a slightly complicated NN

In [ ]:
# import the data set from tensorflow
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data() 
# the data has already been explored before.
# let's consider making a powerful classifier with convolutional neural networks.

In [ ]:
def cnn_fashion_model(input_shape=(28, 28, 1), num_classes=10):
    # define the input
    inputs = tf.keras.Input(shape=input_shape)
    # relu activation is used extensively:
    relu = tfl.ReLU()
    # 2D convolutional layer
    f1 = 16
    size1  = (3, 3)
    conv1 = tfl.Conv2D(filters=f1, kernel_size=size1, strides=(2, 2), padding='same')
    
    pool1 = tfl.MaxPool2D((2, 2))
    f2 = 32
    conv2 = tfl.Conv2D(filters=f2, kernel_size=size1, strides=(1, 1), padding='same')
    
    # normalize the input on the the channels axis
    batnor = tfl.BatchNormalization(axis=3)
    size2 = (3, 3)
    f3 = 64
    conv3 = tfl.Conv2D(filters=f3, kernel_size=size2, strides=(2, 2), padding='same')

    f4 = 64
    conv4 = tfl.Conv2D(filters=f4, kernel_size=size2, strides=(1, 1), padding='same')

    pool2 = tfl.MaxPool2D((2, 2))

    # the neural network should be as follows:
    # con1 * 2 -> conv2 *2 -> pool1 -> conv3 * 2 -> conv4 * 2 -> pool2 -> fully connected1 -> fullyconnected2 -> softmax
    x = conv1(inputs)
    x = relu(x)

    x = conv2(inputs)
    x = relu(x)


    x = tfl.BatchNormalization(axis=3)(x)
    x = pool1(x)
    
    x = conv3(x)
    x = relu(x)

    x = conv4(x)
    x = relu(x)

    x = tfl.BatchNormalization(axis=3)(x)
    x = pool2(x)

    flatten = tfl.Flatten() 
    fc1 = tfl.Dense(128, activation='relu')
    fc2 = tfl.Dense(128, activation='relu')
    fc3 = tfl.Dense(num_classes, activation='softmax')

    x = flatten(x)
    x = fc1(x)
    x = fc2(x)
    outputs = fc3(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
model = cnn_fashion_model()
print(model.summary())

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.002)
metrics = ['accuracy']
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

model.compile(optimizer=optimizer, metrics=metrics, loss=loss)

In [ ]:
model.fit(X_train, y_train, epochs=12)
model.evaluate(X_test, y_test, verbose=2)

In [ ]:
def reshape_and_normalize(images):
    
    new_shape = list(images.shape)
    new_shape.append(1)
    new_shape = tuple(new_shape)
    # Reshape the images to add an extra dimension
    
    images = images.reshape(new_shape)
    
    maximum_value = np.max(images[0])
    # Normalize pixel values
    images = images / maximum_value
    
    ### END CODE HERE

    return images

In [ ]:
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
X_train = reshape_and_normalize(X_train)
y_train = reshape_and_normalize(y_train)
X_test = reshape_and_normalize(X_test)
y_test = reshape_and_normalize(y_test)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshhold=0.995):
        self.threshhold = threshhold

    def on_epoch_end(self, epoch, logs={}): # this method checks the model's status at the end of each epoch
        if (logs.get('accuracy') >= self.threshhold):
            print("Reached {}% accuracy so cancelling training!".format(str(self.threshhold)))
            self.model.stop_training=True



In [ ]:
def convolutional_model(num_classes=10):
    # this architecture consists of one convolutional layer and some fully connected layers
    model = tf.keras.Sequential([
        tfl.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='valid'), 
        tfl.MaxPool2D((2, 2)),
        tfl.Conv2D(filters=64, kernel_size, strides=(1, 1), padding='valid'),
        tfl.MaxPool2D((2, 2)),
        tfl.Conv2D(filters=128, kernel_size=(3, 3), strides=(2, 2), padding='valid')
        tfl.MaxPool2d((2, 2))
        tfl.Flatten(),
        tfl.Dense(128, activation='relu'),
        tfl.Dense(num_classes if num_classes > 2 else 1, activation='softmax' if numclasses > 2 else "sigmoid")
    ])
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy' if num_classes > 2 else "binary_categorical_crossentropy", 
                  metrics=['accuracy']) 
        
    return model


In [ ]:
model = cnn_fashion_model()
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.002)
metrics = ['accuracy']
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

model.compile(optimizer=optimizer, metrics=metrics, loss=loss)
# Instantiate the callback class
callbacks = myCallback()

# Train your model (this can take up to 5 minutes)
history = model.fit(X_train, y_train, epochs=10, callbacks=[callbacks])

In [ ]:
model.evaluate(X_test, y_test)
